In [ ]:
"""
doi:10.1111/j.1365-2966.2012.21831.x

1. exclude the point sources regions
1.1 region command, set the bkg fraction as the variable: np.arange(0.5, 2.5, 0.1)
1.2 get different band images 0.5-2, 2-8, 0.5-8 keV

2. define the bkg region
2.1 get the exp-corr and bkg-sub sb profile (0.5-2, 2-8 keV)
criteria for bkg region: 2-8 keV sb fluctuation < 10% of the bkg level


"""

# 1 Exclude the point sources regions

## 1.1 region command

In [ ]:
## edit region files
# onserver: region_2nd_scripts.py
# then run . region_2nd_all.sh
"""local
# create emllist_final.fits, to convert the centers of fk5 coords to detxy and xy coords
rootdir = '/Users/eusracenorth/Documents/work/XGAP-ABUN/alldata/XGAP/'
datpath = f'{rootdir}/dats/logNS'
dat_f = pd.read_csv(f'{datpath}/readme.csv')

# ids = dat_f['SRCID']
ids = ['SDSSTG40241']
"""
from glob import glob
import os
from astropy.io import fits
from tqdm import tqdm

rootdir = '/data/yanling/XGAP-ABUN/data/alldata/XGAP'
ids = [name.split('/')[-1] for name in glob(f'{rootdir}/SDSSTG*')]

for ID in tqdm(ids): 
    os.chdir(f'{rootdir}/{ID}')
    os.makedirs('cheese_before', exist_ok=True)
    
    f = open(f'region_2nd.sh', 'w')
    f.write(
'''
#!/bin/bash
. ./set_sas.txt
export elo=350
export ehi=8000
''')
    for inst in ['mos1', 'mos2', 'pn']:
        evtfile = glob(f'{inst}*-allevc.fits')[0]
        name = evtfile.split('-')[0]
        f.write(
f'''
region eventset={evtfile} operationstyle=global srclisttab=emllist_final.fits:SRCLIST expression="(ID_INST == 0)&&(DET_ML >= 1)" bkgregionset={name}-bkgregtdet.fits energyfraction=0.9 radiusstyle=enfrac outunit=detxy verbosity=4 2>&1 | tee region-det-{name}_2nd.log
region eventset={evtfile} operationstyle=global srclisttab=emllist_final.fits:SRCLIST expression="(ID_INST == 0)&&(DET_ML >= 1)" bkgregionset={name}-bkgregtsky.fits energyfraction=0.9 radiusstyle=enfrac outunit=xy verbosity=4 2>&1 | tee region-sky-{name}_2nd.log
'''
)
    f.close()

## 1.2.1 alter the radius of the regions

In [ ]:
# the physical coordinates in mos1S001,mos2S002,pnS003-bkgregtsky should be the same
# onserver: alter_bkgfrac_radius_bkgregt.py
""" local
rootdir = '/Users/eusracenorth/Documents/work/XGAP-ABUN/alldata/XGAP/'
datpath = f'{rootdir}/dats/logNS'
dat_f = pd.read_csv(f'{datpath}/readme.csv')
# ids = dat_f['SRCID']
ids = ['SDSSTG40241']
"""
from glob import glob
import os
from astropy.io import fits
from tqdm import tqdm
import numpy as np

rootdir = '/data/yanling/XGAP-ABUN/data/alldata/XGAP'
ids = [name.split('/')[-1] for name in glob(f'{rootdir}/SDSSTG*')]
bkgfracs = np.arange(0.5, 2.6, 0.1)

for ID in tqdm(ids): 
    os.chdir(f'{rootdir}/{ID}')
    for name in ['mos1', 'mos2', 'pn']:
        for bkgfrac in bkgfracs:
            bkgfrac = np.round(bkgfrac,1)
            os.system(f'cp {name}-bkgregtsky.fits {name}-bkgregtsky_{bkgfrac}_09fov.fits')
            data_f = fits.open(f'{name}-bkgregtsky_{bkgfrac}_09fov.fits', mode = 'update')
            dat = data_f[1].data
            dat['R'][:,0] = dat['R'][:,0] * bkgfrac
            dat['R'][:,1] = dat['R'][:,1] * bkgfrac

            data_f.flush()



## 1.2 convert the *bkgregtsky.fits to region file to check

In [1]:
# the physical coordinates in mos1S001,mos2S002,pnS003-bkgregtsky should be the same
# onserver: ps-bkgregt-reg.py, ps-bkgregt-coords.py
""" local
rootdir = '/Users/eusracenorth/Documents/work/XGAP-ABUN/alldata/XGAP/'
datpath = f'{rootdir}/dats/logNS'
dat_f = pd.read_csv(f'{datpath}/readme.csv')
# ids = dat_f['SRCID']
ids = ['SDSSTG40241']
"""
from glob import glob
import os
from astropy.io import fits
from tqdm import tqdm

rootdir = '/data/yanling/XGAP-ABUN/data/alldata/XGAP'
ids = [name.split('/')[-1] for name in glob(f'{rootdir}/SDSSTG*')]
bkgfrac =1 

for ID in tqdm(ids): 
    os.chdir(f'{rootdir}/{ID}')
    for name in ['mos1', 'mos2', 'pn']:
        for file in glob(f'{name}-bkgregtsky_*.*_09fov.fits'):
            data_f = fits.open(file)
            dat = data_f[1].data

            f = open(f'{file.split(".fits")[0]}.reg','w')
            f.write('physical\n')
            for i in range(len(dat)):
                f.write(f'''ellipse({dat[i]['X'][0]},{dat[i]['Y'][0]},{dat[i]['R'][0]},{dat[i]['R'][1]},{dat[i]['ROTANG'][0]})\n''')
            f.close()



0it [00:00, ?it/s]


## 1.3 make mask

In [ ]:
## make mask!
# onserver: makemask_2nd_scripts.py
# onserver: makemask_2nd_all.sh

"""local
rootdir = '/Users/eusracenorth/Documents/work/XGAP-ABUN/alldata/XGAP/'
datpath = f'{rootdir}/dats/logNS'
dat_f = pd.read_csv(f'{datpath}/readme.csv')

ids = dat_f['SRCID']
# ids = ['SDSSTG40241']
"""

from glob import glob
import os
from astropy.io import fits
from tqdm import tqdm
import pandas as pd

rootdir = '/data/yanling/XGAP-ABUN/data/alldata/XGAP'
ids = [name.split('/')[-1] for name in glob(f'{rootdir}/SDSSTG*')]

for ID in tqdm(ids): 
    os.chdir(f'{rootdir}/{ID}')
    f = open(f'makemask_2nd.sh', 'w')
    for name in ['mos1', 'mos2', 'pn']:
        imgfile = glob(f'{name}*-fovimsky-350-2000.fits')[0]
        mskfile = glob(f'{name}*-fovimtmask.fits')[0]
        longname = mskfile.split('-')[0]
        f.write(f'''
makemask imagefile={imgfile} maskfile={mskfile} regionfile={name}-bkgregtsky.fits cheesefile={longname}-cheeset.fits 2>&1 | tee makemask-{name}_2nd.log
        ''')

    f.close()

## 1.4 comb images (go to image_pipeline first)

In [ ]:
## onserver: comb_img_final.py  
## onserver: comb_img_all.sh

def reorder_list(input_list):
    order = ['mos1', 'mos2', 'pn']
    return sorted(input_list, key=lambda x: (order.index(x[:-4]), x[-4:]))

import os
from glob import glob

homepaths = glob('/data/yanling/XGAP-ABUN/data/alldata/XGAP/SDSSTG*')

for homepath in homepaths:
    if os.path.isdir(homepath):
        os.chdir(homepath)
        f = open('comb_img.sh', 'w')

        elo = 500
        ehi = 2000
        bkgfrac = 1

        name_dir = {'mos1':'mos1*-allevc', 'mos2':'mos2*-allevc', 'pn':'pn*-allevc'}

        express = f'(PATTERN<=4)&&((FLAG & 0xb0000) == 0)&&((FLAG & 0x762a097c) == 0)&&(PI in [{elo}:{ehi}])&&(DETY in [-16510:14345])'
        plot_express = f'''imagebinning='imageSize' imagedatatype='Int32' squarepixels=yes ignorelegallimits=yes withxranges=yes withyranges=yes xcolumn='X' ximagesize=900 ximagemax=48400 ximagemin=3401 ycolumn='Y' yimagesize=900 yimagemax=48400 yimagemin=3401'''

        name_lst = []
        prefix_lst = ''
        for file in glob(f'*0*-allevc.fits'):
            prefix = file.split('-')[0]
            name_lst.append(prefix)
        name_lst = reorder_list(name_lst)
        for prefix in name_lst:
            if 'mos' in prefix:
                prefix_lst += f"{prefix[-5:]}  "
            else:
                prefix_lst += f"{prefix[-4:]}"

        for name in name_lst:
            f.write(f'''source set_sas.txt \n''')
            f.write(f'''source set_chips_on.txt \n''')
            f.write(f'''evselect table={name}-allevc.fits withimageset=Y {plot_express} expression='{express}' imageset='{name}-fovimsky-{elo}-{ehi}.fits'  \n''')
            if 'pn' in name:
                f.write(f'''evselect table={name}-allevcoot.fits withimageset=Y {plot_express} expression='{express}' imageset='{name}-fovimootsky-{elo}-{ehi}.fits'  \n''')
            f.write(f'mv {name}-fovimt.fits {name}-fovimsky-{elo}-{ehi}.fits\n')
            f.write(f'eexpmap imageset={name}-fovimsky-{elo}-{ehi}.fits attitudeset=atthk.fits eventset={name}-allevc.fits pimin={elo} pimax={ehi} expimageset={name}-expimsky-{elo}-{ehi}.fits  2>&1 | tee create_expmaps.log\n')
            # f.write(f'cp {name[:-4]}-cheeset.fits {name}-cheeset.fits\n')

        f.write(f'''combimage prefixlist="{prefix_lst}" withcheese=yes cheesetype=t elowlist={elo} ehighlist={ehi} > combimg_{elo}-{ehi}_{bkgfrac}_09psf.log 2>&1 \n''')
        f.write(f'''mv comb-fovimsky-{elo}-{ehi}.fits comb-fovimsky-{elo}-{ehi}_masked_{bkgfrac}_09psf.fits\n''')
        f.write(f'''mv comb-expimsky-{elo}-{ehi}.fits comb-expimsky-{elo}-{ehi}_masked_{bkgfrac}_09psf.fits\n''')
        f.write(f'farith comb-fovimsky-{elo}-{ehi}_masked.fits comb-expimsky-{elo}-{ehi}_masked.fits comb-ctr-{elo}-{ehi}_masked_{bkgfrac}_09psf.fits / \n')
        f.write(f'fimgtrim comb-ctr-{elo}-{ehi}_masked.fits threshlo=0 threshup=1e-3 const_lo=0 const_up=0 outfile=comb-ctr-{elo}-{ehi}_masked_trimmed_{bkgfrac}_09psf.fits clobber=yes\n')

        # f.write(f'''combimage prefixlist="{prefix_lst}" withcheese=no elowlist={elo} ehighlist={ehi} > -a combimg.log 2>&1 \n''')
        # f.write(f'''mv comb-fovimsky-{elo}-{ehi}.fits comb-fovimsky-{elo}-{ehi}_unmasked.fits\n''')
        # f.write(f'''mv comb-expimsky-{elo}-{ehi}.fits comb-expimsky-{elo}-{ehi}_unmasked.fits\n''')
        
        f.close()

# 2 

## 2.1 go to image_pipeline.ipynb